In [2]:
import sys
# sys.path.insert(0, '../../Utilities/')
from mpl_toolkits import mplot3d
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
#from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time
# tf.enable_eager_execution()
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [33]:
class PhysicsInformedNN2:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub , q):
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
#         self.t = X[:,1:2]
        self.u = u
        
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))

        # Initialize parameters
        self.lambda_1 = tf.Variable([6.0], dtype=tf.float32)
#         self.lambda_2 = tf.Variable([-6.0], dtype=tf.float32)
        self.q = q
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
#         self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
                
        self.u_pred = self.net_u(self.x_tf)
        self.f_pred = self.net_f(self.x_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
#         self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
#                                                                 method = 'L-BFGS-B', 
#                                                                 options = {'maxiter': 50000,
#                                                                            'maxfun': 50000,
#                                                                            'maxcor': 50,
#                                                                            'maxls': 50,
#                                                                            'ftol' : 1.0 * np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = (X - self.lb)/(self.ub - self.lb)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
#             H = tf.math.sigmoid(tf.add(tf.matmul(H, W), b))
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
        
            
    def net_u(self, x):  
        u = self.neural_net(tf.concat([x],1), self.weights, self.biases)
        return u
    
    def net_f(self, x):
        lambda_1 = self.lambda_1       
#         lambda_2 = tf.exp(self.lambda_2)
        q= self.q
        u = self.net_u(x)
        u_x = tf.gradients(u, x)[0]
#         u_xx = tf.gradients(u_x, x)[0]
        f= q + lambda_1*u_x
#         f = -1*lambda_1*u_xx 
#         f = -1*lambda_1*u_xx
        
        return f
    
    def callback(self, loss, lambda_1):
        print('Loss: %e, l1: %.5f' % (loss, lambda_1))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x, self.u_tf: self.u}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
#                 lambda_2_value = np.exp(self.sess.run(self.lambda_2))
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, elapsed))
#                 lolz= self.sess.run(self.net_f(self.x_tf) , tf_dict)
#                 u_pred_xx = tf.gradients(u_pred_x,self.sess.run(self.x_tf,tf_dict))[0]
                
#                 print(lolz)
#                 print((self.net_f(self.sess.run(self.x_tf,tf_dict))))
                
#                 print(self.sess.run(self.u_tf,tf_dict) ," " , self.sess.run(self.u_pred,tf_dict))
                start_time = time.time()
        
#         self.optimizer.minimize(self.sess,
#                                 feed_dict = tf_dict,
#                                 fetches = [self.loss, self.lambda_1],
#                                 loss_callback = self.callback)
#         lolz= self.sess.run(self.net_f(self.x_tf) , tf_dict)
#         sike= self.sess.run(self.net_u(self.x_tf),tf_dict)
#         print(lolz)
#         print(sike)
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [3]:
df= pd.read_csv(r"C:\Users\Omkar\Downloads\K5(1800,1000,300,500,CCW,start from left side of square)_.csv")

In [4]:
df

,X Location (m),Z Location (m),Temperature (°C),Directional Heat Flux (Z),Directional Heat Flux (X)
0,1.000,1.000,1000.00,-1.027400e-08,8.000000e+05
1,1.000,0.000,1000.00,-2.145700e-10,-2.234800e+02
2,0.000,0.000,900.00,-3.134000e+03,-1.039200e+03
3,0.000,1.000,1800.00,-5.678100e+03,-1.607100e-10
4,0.005,0.005,904.17,-3.138100e+03,-1.034900e+03
...,...,...,...,...,...
160796,-0.975,-1.000,300.00,-2.464600e+04,1.476800e-10
160797,-0.980,-1.000,300.00,-3.025600e+04,1.707600e-10
160798,-0.985,-1.000,300.00,-3.901800e+04,3.397000e-10
160799,-0.990,-1.000,300.00,-5.440200e+04,5.579700e-10


In [4]:
df2=df[ ( (df['X Location (m)'] >= -0.8) & (df['X Location (m)'] <= 0.8) ) & ( (df['Z Location (m)'] >= -0.8) & (df['Z Location (m)'] <= 0.8) ) ]

In [7]:
df2

,X Location (m),Z Location (m),Temperature (°C),Directional Heat Flux (Z),Directional Heat Flux (X)
2,0.000,0.000,900.00,-3134.0,-1039.2
4,0.005,0.005,904.17,-3138.1,-1034.9
5,0.010,0.005,905.21,-3137.8,-1026.7
6,0.015,0.005,906.23,-3137.4,-1018.4
7,0.020,0.005,907.24,-3136.9,-1010.3
...,...,...,...,...,...
152637,-0.800,-0.025,601.78,-1092.2,-2466.5
152638,-0.800,-0.020,602.87,-1098.0,-2492.2
152639,-0.800,-0.015,603.98,-1103.8,-2517.9
152640,-0.800,-0.010,605.08,-1109.8,-2543.8


In [5]:
q_z= df2.loc[:,["Directional Heat Flux (Z)" ]] 

In [6]:
q_z

,Directional Heat Flux (Z)
0,-1.027400e-08
1,-2.145700e-10
2,-3.134000e+03
3,-5.678100e+03
4,-3.138100e+03
...,...
160796,-2.464600e+04
160797,-3.025600e+04
160798,-3.901800e+04
160799,-5.440200e+04


In [6]:
q_z_new=np.array(q_z)

In [7]:
q_z_squared=np.square(q_z_new)

In [8]:
q_x= df2.loc[:,["Directional Heat Flux (X)" ]] 

In [9]:
q_x_new=np.array(q_x)

In [48]:
q_x_s=q_x_new/10000

In [10]:
q_x_squared=np.square(q_x_new)

In [11]:
q_final = np.sqrt((q_z_squared+q_x_squared))

In [12]:
q_final

array([[3301.80142347],
       [3304.34405291],
       [3301.49992125],
       ...,
       [2749.21713402],
       [2775.35123543],
       [2801.6253943 ]])

In [27]:
# scaler = MinMaxScaler()
# model4 = scaler.fit(q_x_new)
# scaled_q = model4.transform(q_x_new)
scaled_q=q_final/10000

In [29]:
scaled_q

array([[0.33018014],
       [0.33043441],
       [0.33014999],
       ...,
       [0.27492171],
       [0.27753512],
       [0.28016254]])

In [12]:
x = df2.loc[:,["X Location (m)"]]

In [13]:
x=np.array(x)

In [14]:
z = df2.loc[:,["Z Location (m)"]]

In [15]:
z=np.array(z)

In [28]:
scaler = MinMaxScaler()
model2 = scaler.fit(z)
scaled_x = model2.transform(z)

In [30]:
x_y=df2.loc[:,["X Location (m)","Z Location (m)" ]] 

In [31]:
x_y=np.array(x_y)

In [38]:
# scaled_x_y=x_y/5000

In [10]:
scaler = MinMaxScaler()
model2 = scaler.fit(x_y)
scaled_x_y = model2.transform(x_y)

In [32]:
u=df2.loc[:,"Temperature (°C)"]

In [33]:
u=np.array(u)

In [34]:
u=u.reshape(103041,1)

In [35]:
# scaler = MinMaxScaler()
# model1 = scaler.fit(u)
# scaled_u = model1.transform(u)

scaled_u = u/10000

In [23]:
N_u_1=100000
layers1 = [1, 20, 20, 20, 20, 20, 20, 20, 20, 1]

In [30]:
lb1 = scaled_x.min(0)
ub1 = scaled_x.max(0)


In [49]:
idx = np.random.choice(scaled_x.shape[0], N_u_1, replace=False)
X_u_train = scaled_x[idx,:]
u_train = scaled_u[idx,:]
# q = scaled_q[idx,:]
q_= q_x_s[idx,:]
q_t= tf.convert_to_tensor(q_,dtype=tf.float32)
# q_= tf.convert_to_tensor(q,dtype=tf.float32)



NameError: name 'scaled_x' is not defined

In [34]:
model = PhysicsInformedNN2(X_u_train, u_train, layers1, lb, ub , q_t)
model.train(10000)

Device mapping:

It: 0, Loss: 8.636e-01, Lambda_1: 5.999, Time: 0.97
It: 10, Loss: 2.540e-01, Lambda_1: 5.995, Time: 3.81
It: 20, Loss: 1.696e-01, Lambda_1: 5.991, Time: 3.60
It: 30, Loss: 1.070e-01, Lambda_1: 5.992, Time: 3.40
It: 40, Loss: 9.181e-02, Lambda_1: 5.992, Time: 3.28
It: 50, Loss: 9.045e-02, Lambda_1: 5.992, Time: 3.23
It: 60, Loss: 9.011e-02, Lambda_1: 5.993, Time: 3.30
It: 70, Loss: 8.938e-02, Lambda_1: 5.994, Time: 3.34
It: 80, Loss: 8.902e-02, Lambda_1: 5.995, Time: 2.76
It: 90, Loss: 8.886e-02, Lambda_1: 5.995, Time: 2.51
It: 100, Loss: 8.871e-02, Lambda_1: 5.996, Time: 2.48
It: 110, Loss: 8.861e-02, Lambda_1: 5.998, Time: 2.50
It: 120, Loss: 8.853e-02, Lambda_1: 5.999, Time: 2.45
It: 130, Loss: 8.847e-02, Lambda_1: 6.000, Time: 2.49
It: 140, Loss: 8.842e-02, Lambda_1: 6.001, Time: 2.53
It: 150, Loss: 8.838e-02, Lambda_1: 6.002, Time: 2.56
It: 160, Loss: 8.834e-02, Lambda_1: 6.004, Time: 2.61
It: 170, Loss: 8.829e-02, Lambda_1: 6.005, Time: 2.58
It: 180, Loss: 8.825e-

KeyboardInterrupt: 

In [36]:
N_u_2=3000
layers2 = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

In [37]:
lb = x_y.min(0)
ub = x_y.max(0)

In [38]:
idx = np.random.choice(x_y.shape[0], N_u_2, replace=False)
X_u_train = x_y[idx,:]
u_train = scaled_u[idx,:]
# q = scaled_q[idx,:]
q_= scaled_q[idx,:]
q_t= tf.convert_to_tensor(q_,dtype=tf.float32)
# q_= tf.convert_to_tensor(q,dtype=tf.float32)

In [41]:
class PhysicsInformedNN3:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub, q):
        
        
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        self.u = u
        
        self.layers = layers
        self.q = q
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        # Initialize parameters
        self.lambda_1 = tf.Variable([5.5], dtype=tf.float32)
        self.lambda_2 = tf.Variable([1.0], dtype=tf.float32)
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
                
        self.u_pred = self.net_u(self.x_tf, self.t_tf)
        self.f_pred = self.net_f(self.x_tf, self.t_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
    
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = (X - self.lb)/(self.ub - self.lb) 
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.math.sigmoid(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
            
    def net_u(self, x, t):  
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        return u
    
    def net_f(self, x, t):
        lambda_1 = self.lambda_1        
#         lambda_2 = self.lambda_2
        q = self.q 
        u = self.net_u(x,t)
        u_t = tf.gradients(u, t)[0]
#         u_tt = tf.gradients(u_t, t)[0]
        u_x = tf.gradients(u, x)[0]
#         u_xx = tf.gradients(u_x, x)[0]
#         f = q + lambda_1*u_x 
        f = q - lambda_1*(tf.math.sqrt(tf.math.square(u_x) + tf.math.square(u_t))) 
        
        return f
    
    def callback(self, loss, lambda_1, lambda_2):
        print('Loss: %e, l1: %.5f, l2: %.5f' % (loss, lambda_1, lambda_2))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
                lambda_2_value = self.sess.run(self.lambda_2)
#                 lolz= self.sess.run(self.net_f(self.x_tf,self.t_tf) , tf_dict)
                print('It: %d, Loss: %.3e, Lambda_1: %.3f, Lambda_2: %.6f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, lambda_2_value, elapsed))
#                 print(lolz)
                start_time = time.time()
        
#         self.optimizer_Adam.minimize(self.loss)
        
        
#         self.optimizer.minimize(self.sess,
#                                 feed_dict = tf_dict,
#                                 fetches = [self.loss, self.lambda_1, self.lambda_2],
#                                 loss_callback = self.callback)
        
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star[:,0:1], self.t_tf: X_star[:,1:2]}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [40]:
model = PhysicsInformedNN3(X_u_train, u_train, layers2, lb, ub , q_t)
model.train(20000)

Device mapping:

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

It: 0, Loss: 1.873e-01, Lambda_1: 6.001, Lambda_2: 1.000000, Time: 0.35
It: 10, Loss: 1.665e-01, Lambda_1: 6.008, Lambda_2: 1.000000, Time: 0.12
It: 20, Loss: 1.657e-01, Lambda_1: 6.018, Lambda_2: 1.000000, Time: 0.11
It: 30, Loss: 1.655e-01, Lambda_1: 6.030, Lambda_2: 1.000000, Time: 0.09
It: 40, Loss: 1.649e-01, Lambda_1: 6.043, Lambda_2: 1.000000, Time: 0.08
It: 50, Loss: 1.650e-01, Lambda_1: 6.057, Lambda_2: 1.000000, Time: 0.09
It: 60, Loss: 1.649e-01, Lambda_1: 6.072, Lambda_2: 1.000000, Time: 0.09
It: 70, Loss: 1.649e-01, Lambda_1: 6.089, Lambda_2: 1.000000, Time: 0.08
It: 80, Loss: 1.648e-01, Lambda_1: 6

It: 1060, Loss: 1.073e-02, Lambda_1: 6.450, Lambda_2: 1.000000, Time: 0.11
It: 1070, Loss: 1.063e-02, Lambda_1: 6.451, Lambda_2: 1.000000, Time: 0.10
It: 1080, Loss: 1.052e-02, Lambda_1: 6.452, Lambda_2: 1.000000, Time: 0.10
It: 1090, Loss: 1.039e-02, Lambda_1: 6.452, Lambda_2: 1.000000, Time: 0.10
It: 1100, Loss: 1.025e-02, Lambda_1: 6.453, Lambda_2: 1.000000, Time: 0.09
It: 1110, Loss: 1.008e-02, Lambda_1: 6.454, Lambda_2: 1.000000, Time: 0.10
It: 1120, Loss: 9.892e-03, Lambda_1: 6.454, Lambda_2: 1.000000, Time: 0.09
It: 1130, Loss: 9.669e-03, Lambda_1: 6.455, Lambda_2: 1.000000, Time: 0.10
It: 1140, Loss: 9.408e-03, Lambda_1: 6.456, Lambda_2: 1.000000, Time: 0.10
It: 1150, Loss: 9.102e-03, Lambda_1: 6.456, Lambda_2: 1.000000, Time: 0.10
It: 1160, Loss: 8.744e-03, Lambda_1: 6.457, Lambda_2: 1.000000, Time: 0.10
It: 1170, Loss: 8.326e-03, Lambda_1: 6.458, Lambda_2: 1.000000, Time: 0.10
It: 1180, Loss: 7.849e-03, Lambda_1: 6.458, Lambda_2: 1.000000, Time: 0.10
It: 1190, Loss: 7.318e-03

It: 2170, Loss: 3.010e-03, Lambda_1: 6.511, Lambda_2: 1.000000, Time: 0.08
It: 2180, Loss: 3.008e-03, Lambda_1: 6.512, Lambda_2: 1.000000, Time: 0.08
It: 2190, Loss: 3.006e-03, Lambda_1: 6.512, Lambda_2: 1.000000, Time: 0.08
It: 2200, Loss: 3.004e-03, Lambda_1: 6.513, Lambda_2: 1.000000, Time: 0.08
It: 2210, Loss: 3.002e-03, Lambda_1: 6.513, Lambda_2: 1.000000, Time: 0.08
It: 2220, Loss: 3.000e-03, Lambda_1: 6.514, Lambda_2: 1.000000, Time: 0.08
It: 2230, Loss: 2.998e-03, Lambda_1: 6.514, Lambda_2: 1.000000, Time: 0.08
It: 2240, Loss: 2.996e-03, Lambda_1: 6.515, Lambda_2: 1.000000, Time: 0.08
It: 2250, Loss: 2.994e-03, Lambda_1: 6.515, Lambda_2: 1.000000, Time: 0.08
It: 2260, Loss: 2.992e-03, Lambda_1: 6.516, Lambda_2: 1.000000, Time: 0.09
It: 2270, Loss: 2.991e-03, Lambda_1: 6.516, Lambda_2: 1.000000, Time: 0.09
It: 2280, Loss: 2.989e-03, Lambda_1: 6.517, Lambda_2: 1.000000, Time: 0.09
It: 2290, Loss: 2.987e-03, Lambda_1: 6.518, Lambda_2: 1.000000, Time: 0.09
It: 2300, Loss: 2.985e-03

It: 3280, Loss: 2.717e-03, Lambda_1: 6.570, Lambda_2: 1.000000, Time: 0.09
It: 3290, Loss: 2.713e-03, Lambda_1: 6.570, Lambda_2: 1.000000, Time: 0.08
It: 3300, Loss: 2.709e-03, Lambda_1: 6.571, Lambda_2: 1.000000, Time: 0.09
It: 3310, Loss: 2.719e-03, Lambda_1: 6.571, Lambda_2: 1.000000, Time: 0.09
It: 3320, Loss: 2.708e-03, Lambda_1: 6.572, Lambda_2: 1.000000, Time: 0.09
It: 3330, Loss: 2.699e-03, Lambda_1: 6.572, Lambda_2: 1.000000, Time: 0.09
It: 3340, Loss: 2.694e-03, Lambda_1: 6.573, Lambda_2: 1.000000, Time: 0.10
It: 3350, Loss: 2.689e-03, Lambda_1: 6.573, Lambda_2: 1.000000, Time: 0.10
It: 3360, Loss: 2.685e-03, Lambda_1: 6.574, Lambda_2: 1.000000, Time: 0.10
It: 3370, Loss: 2.681e-03, Lambda_1: 6.574, Lambda_2: 1.000000, Time: 0.10
It: 3380, Loss: 2.677e-03, Lambda_1: 6.574, Lambda_2: 1.000000, Time: 0.09
It: 3390, Loss: 2.673e-03, Lambda_1: 6.575, Lambda_2: 1.000000, Time: 0.09
It: 3400, Loss: 2.669e-03, Lambda_1: 6.575, Lambda_2: 1.000000, Time: 0.09
It: 3410, Loss: 2.664e-03

It: 4400, Loss: 1.817e-03, Lambda_1: 6.658, Lambda_2: 1.000000, Time: 0.10
It: 4410, Loss: 1.810e-03, Lambda_1: 6.660, Lambda_2: 1.000000, Time: 0.10
It: 4420, Loss: 1.804e-03, Lambda_1: 6.663, Lambda_2: 1.000000, Time: 0.10
It: 4430, Loss: 1.799e-03, Lambda_1: 6.665, Lambda_2: 1.000000, Time: 0.10
It: 4440, Loss: 1.804e-03, Lambda_1: 6.667, Lambda_2: 1.000000, Time: 0.10
It: 4450, Loss: 1.795e-03, Lambda_1: 6.669, Lambda_2: 1.000000, Time: 0.10
It: 4460, Loss: 1.783e-03, Lambda_1: 6.671, Lambda_2: 1.000000, Time: 0.10
It: 4470, Loss: 1.776e-03, Lambda_1: 6.673, Lambda_2: 1.000000, Time: 0.09
It: 4480, Loss: 1.771e-03, Lambda_1: 6.676, Lambda_2: 1.000000, Time: 0.09
It: 4490, Loss: 1.767e-03, Lambda_1: 6.678, Lambda_2: 1.000000, Time: 0.09
It: 4500, Loss: 1.762e-03, Lambda_1: 6.680, Lambda_2: 1.000000, Time: 0.09
It: 4510, Loss: 1.757e-03, Lambda_1: 6.682, Lambda_2: 1.000000, Time: 0.10
It: 4520, Loss: 1.752e-03, Lambda_1: 6.685, Lambda_2: 1.000000, Time: 0.10
It: 4530, Loss: 1.748e-03

...

It: 15300, Loss: 2.504e-05, Lambda_1: 5.142, Lambda_2: 1.000000, Time: 0.18
It: 15310, Loss: 2.573e-05, Lambda_1: 5.141, Lambda_2: 1.000000, Time: 0.18
It: 15320, Loss: 1.221e-04, Lambda_1: 5.141, Lambda_2: 1.000000, Time: 0.17
It: 15330, Loss: 2.673e-05, Lambda_1: 5.141, Lambda_2: 1.000000, Time: 0.16
It: 15340, Loss: 2.597e-05, Lambda_1: 5.141, Lambda_2: 1.000000, Time: 0.17
It: 15350, Loss: 2.519e-05, Lambda_1: 5.140, Lambda_2: 1.000000, Time: 0.18
It: 15360, Loss: 2.515e-05, Lambda_1: 5.140, Lambda_2: 1.000000, Time: 0.19
It: 15370, Loss: 2.500e-05, Lambda_1: 5.140, Lambda_2: 1.000000, Time: 0.19
It: 15380, Loss: 2.496e-05, Lambda_1: 5.140, Lambda_2: 1.000000, Time: 0.19
It: 15390, Loss: 2.493e-05, Lambda_1: 5.139, Lambda_2: 1.000000, Time: 0.18
It: 15400, Loss: 2.488e-05, Lambda_1: 5.139, Lambda_2: 1.000000, Time: 0.18
It: 15410, Loss: 2.484e-05, Lambda_1: 5.139, Lambda_2: 1.000000, Time: 0.18
It: 15420, Loss: 2.482e-05, Lambda_1: 5.139, Lambda_2: 1.000000, Time: 0.19
It: 15430, L

It: 16380, Loss: 2.313e-05, Lambda_1: 5.117, Lambda_2: 1.000000, Time: 0.25
It: 16390, Loss: 2.312e-05, Lambda_1: 5.117, Lambda_2: 1.000000, Time: 0.24
It: 16400, Loss: 2.310e-05, Lambda_1: 5.117, Lambda_2: 1.000000, Time: 0.27
It: 16410, Loss: 2.308e-05, Lambda_1: 5.117, Lambda_2: 1.000000, Time: 0.33
It: 16420, Loss: 2.307e-05, Lambda_1: 5.116, Lambda_2: 1.000000, Time: 0.23
It: 16430, Loss: 2.305e-05, Lambda_1: 5.116, Lambda_2: 1.000000, Time: 0.24
It: 16440, Loss: 2.303e-05, Lambda_1: 5.116, Lambda_2: 1.000000, Time: 0.25
It: 16450, Loss: 2.302e-05, Lambda_1: 5.116, Lambda_2: 1.000000, Time: 0.27
It: 16460, Loss: 2.300e-05, Lambda_1: 5.116, Lambda_2: 1.000000, Time: 0.24
It: 16470, Loss: 2.298e-05, Lambda_1: 5.115, Lambda_2: 1.000000, Time: 0.24
It: 16480, Loss: 2.297e-05, Lambda_1: 5.115, Lambda_2: 1.000000, Time: 0.25
It: 16490, Loss: 2.303e-05, Lambda_1: 5.115, Lambda_2: 1.000000, Time: 0.26
It: 16500, Loss: 4.648e-05, Lambda_1: 5.115, Lambda_2: 1.000000, Time: 0.24
It: 16510, L

It: 17460, Loss: 2.167e-05, Lambda_1: 5.100, Lambda_2: 1.000000, Time: 0.24
It: 17470, Loss: 2.159e-05, Lambda_1: 5.100, Lambda_2: 1.000000, Time: 0.24
It: 17480, Loss: 2.152e-05, Lambda_1: 5.100, Lambda_2: 1.000000, Time: 0.32
It: 17490, Loss: 2.149e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.23
It: 17500, Loss: 2.148e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.23
It: 17510, Loss: 2.147e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.25
It: 17520, Loss: 2.145e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.27
It: 17530, Loss: 2.144e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.27
It: 17540, Loss: 2.142e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.26
It: 17550, Loss: 2.141e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.36
It: 17560, Loss: 2.140e-05, Lambda_1: 5.099, Lambda_2: 1.000000, Time: 0.30
It: 17570, Loss: 2.138e-05, Lambda_1: 5.098, Lambda_2: 1.000000, Time: 0.26
It: 17580, Loss: 2.137e-05, Lambda_1: 5.098, Lambda_2: 1.000000, Time: 0.24
It: 17590, L

It: 18540, Loss: 2.335e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.32
It: 18550, Loss: 2.067e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.31
It: 18560, Loss: 2.554e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.29
It: 18570, Loss: 3.502e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.32
It: 18580, Loss: 2.519e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.31
It: 18590, Loss: 2.140e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.26
It: 18600, Loss: 2.015e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.25
It: 18610, Loss: 2.017e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.26
It: 18620, Loss: 2.016e-05, Lambda_1: 5.087, Lambda_2: 1.000000, Time: 0.24
It: 18630, Loss: 2.010e-05, Lambda_1: 5.086, Lambda_2: 1.000000, Time: 0.23
It: 18640, Loss: 2.007e-05, Lambda_1: 5.086, Lambda_2: 1.000000, Time: 0.23
It: 18650, Loss: 2.042e-05, Lambda_1: 5.086, Lambda_2: 1.000000, Time: 0.25
It: 18660, Loss: 2.941e-05, Lambda_1: 5.086, Lambda_2: 1.000000, Time: 0.22
It: 18670, L

It: 19620, Loss: 1.890e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.27
It: 19630, Loss: 1.889e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.27
It: 19640, Loss: 1.888e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.28
It: 19650, Loss: 1.887e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.31
It: 19660, Loss: 1.886e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.26
It: 19670, Loss: 1.885e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.24
It: 19680, Loss: 1.883e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.24
It: 19690, Loss: 1.882e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.26
It: 19700, Loss: 1.881e-05, Lambda_1: 5.078, Lambda_2: 1.000000, Time: 0.24
It: 19710, Loss: 1.880e-05, Lambda_1: 5.077, Lambda_2: 1.000000, Time: 0.23
It: 19720, Loss: 1.879e-05, Lambda_1: 5.077, Lambda_2: 1.000000, Time: 0.24
It: 19730, Loss: 1.878e-05, Lambda_1: 5.077, Lambda_2: 1.000000, Time: 0.26
It: 19740, Loss: 1.877e-05, Lambda_1: 5.077, Lambda_2: 1.000000, Time: 0.26
It: 19750, L

In [42]:
model = PhysicsInformedNN3(X_u_train, u_train, layers2, lb, ub , q_t)
model.train(20000)

Device mapping:

It: 0, Loss: 4.315e-01, Lambda_1: 5.501, Lambda_2: 1.000000, Time: 0.39
It: 10, Loss: 2.425e-01, Lambda_1: 5.509, Lambda_2: 1.000000, Time: 0.12
It: 20, Loss: 1.718e-01, Lambda_1: 5.518, Lambda_2: 1.000000, Time: 0.12
It: 30, Loss: 1.660e-01, Lambda_1: 5.528, Lambda_2: 1.000000, Time: 0.09
It: 40, Loss: 1.683e-01, Lambda_1: 5.538, Lambda_2: 1.000000, Time: 0.09
It: 50, Loss: 1.658e-01, Lambda_1: 5.549, Lambda_2: 1.000000, Time: 0.09
It: 60, Loss: 1.649e-01, Lambda_1: 5.562, Lambda_2: 1.000000, Time: 0.09
It: 70, Loss: 1.651e-01, Lambda_1: 5.576, Lambda_2: 1.000000, Time: 0.09
It: 80, Loss: 1.650e-01, Lambda_1: 5.591, Lambda_2: 1.000000, Time: 0.09
It: 90, Loss: 1.649e-01, Lambda_1: 5.607, Lambda_2: 1.000000, Time: 0.09
It: 100, Loss: 1.649e-01, Lambda_1: 5.623, Lambda_2: 1.000000, Time: 0.09
It: 110, Loss: 1.649e-01, Lambda_1: 5.641, Lambda_2: 1.000000, Time: 0.10
It: 120, Loss: 1.649e-01, Lambda_1: 5.659, Lambda_2: 1.000000, Time: 0.10
It: 130, Loss: 1.649e-01, Lambda

It: 1110, Loss: 2.877e-03, Lambda_1: 6.148, Lambda_2: 1.000000, Time: 0.24
It: 1120, Loss: 2.818e-03, Lambda_1: 6.147, Lambda_2: 1.000000, Time: 0.22
It: 1130, Loss: 2.760e-03, Lambda_1: 6.147, Lambda_2: 1.000000, Time: 0.22
It: 1140, Loss: 2.703e-03, Lambda_1: 6.147, Lambda_2: 1.000000, Time: 0.23
It: 1150, Loss: 2.648e-03, Lambda_1: 6.146, Lambda_2: 1.000000, Time: 0.22
It: 1160, Loss: 2.594e-03, Lambda_1: 6.146, Lambda_2: 1.000000, Time: 0.22
It: 1170, Loss: 2.542e-03, Lambda_1: 6.146, Lambda_2: 1.000000, Time: 0.24
It: 1180, Loss: 2.492e-03, Lambda_1: 6.145, Lambda_2: 1.000000, Time: 0.22
It: 1190, Loss: 2.443e-03, Lambda_1: 6.145, Lambda_2: 1.000000, Time: 0.22
It: 1200, Loss: 2.396e-03, Lambda_1: 6.145, Lambda_2: 1.000000, Time: 0.22
It: 1210, Loss: 2.351e-03, Lambda_1: 6.145, Lambda_2: 1.000000, Time: 0.22
It: 1220, Loss: 2.307e-03, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 1230, Loss: 2.265e-03, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 1240, Loss: 2.225e-03

It: 2210, Loss: 9.514e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2220, Loss: 9.387e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.23
It: 2230, Loss: 9.300e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.24
It: 2240, Loss: 9.228e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.24
It: 2250, Loss: 9.163e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2260, Loss: 9.102e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.24
It: 2270, Loss: 9.044e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2280, Loss: 8.988e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2290, Loss: 8.933e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2300, Loss: 8.880e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2310, Loss: 8.829e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2320, Loss: 8.779e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2330, Loss: 8.731e-04, Lambda_1: 6.144, Lambda_2: 1.000000, Time: 0.22
It: 2340, Loss: 8.685e-04

It: 3310, Loss: 6.499e-04, Lambda_1: 6.126, Lambda_2: 1.000000, Time: 0.22
It: 3320, Loss: 6.481e-04, Lambda_1: 6.126, Lambda_2: 1.000000, Time: 0.22
It: 3330, Loss: 6.476e-04, Lambda_1: 6.126, Lambda_2: 1.000000, Time: 0.22
It: 3340, Loss: 6.857e-04, Lambda_1: 6.126, Lambda_2: 1.000000, Time: 0.23
It: 3350, Loss: 6.604e-04, Lambda_1: 6.125, Lambda_2: 1.000000, Time: 0.22
It: 3360, Loss: 6.466e-04, Lambda_1: 6.125, Lambda_2: 1.000000, Time: 0.22
It: 3370, Loss: 6.407e-04, Lambda_1: 6.125, Lambda_2: 1.000000, Time: 0.22
It: 3380, Loss: 6.379e-04, Lambda_1: 6.125, Lambda_2: 1.000000, Time: 0.23
It: 3390, Loss: 6.360e-04, Lambda_1: 6.124, Lambda_2: 1.000000, Time: 0.22
It: 3400, Loss: 6.343e-04, Lambda_1: 6.124, Lambda_2: 1.000000, Time: 0.22
It: 3410, Loss: 6.325e-04, Lambda_1: 6.124, Lambda_2: 1.000000, Time: 0.22
It: 3420, Loss: 6.308e-04, Lambda_1: 6.123, Lambda_2: 1.000000, Time: 0.22
It: 3430, Loss: 6.291e-04, Lambda_1: 6.123, Lambda_2: 1.000000, Time: 0.22
It: 3440, Loss: 6.273e-04

It: 4410, Loss: 4.783e-04, Lambda_1: 6.084, Lambda_2: 1.000000, Time: 0.22
It: 4420, Loss: 4.773e-04, Lambda_1: 6.083, Lambda_2: 1.000000, Time: 0.22
It: 4430, Loss: 4.763e-04, Lambda_1: 6.083, Lambda_2: 1.000000, Time: 0.22
It: 4440, Loss: 4.752e-04, Lambda_1: 6.082, Lambda_2: 1.000000, Time: 0.23
It: 4450, Loss: 4.742e-04, Lambda_1: 6.082, Lambda_2: 1.000000, Time: 0.22
It: 4460, Loss: 4.732e-04, Lambda_1: 6.081, Lambda_2: 1.000000, Time: 0.22
It: 4470, Loss: 4.721e-04, Lambda_1: 6.081, Lambda_2: 1.000000, Time: 0.22
It: 4480, Loss: 4.711e-04, Lambda_1: 6.080, Lambda_2: 1.000000, Time: 0.22
It: 4490, Loss: 4.702e-04, Lambda_1: 6.080, Lambda_2: 1.000000, Time: 0.22
It: 4500, Loss: 4.692e-04, Lambda_1: 6.079, Lambda_2: 1.000000, Time: 0.22
It: 4510, Loss: 4.682e-04, Lambda_1: 6.079, Lambda_2: 1.000000, Time: 0.22
It: 4520, Loss: 4.672e-04, Lambda_1: 6.078, Lambda_2: 1.000000, Time: 0.23
It: 4530, Loss: 4.663e-04, Lambda_1: 6.078, Lambda_2: 1.000000, Time: 0.22
It: 4540, Loss: 4.653e-04

...

It: 15310, Loss: 8.351e-05, Lambda_1: 5.488, Lambda_2: 1.000000, Time: 0.23
It: 15320, Loss: 8.351e-05, Lambda_1: 5.487, Lambda_2: 1.000000, Time: 0.23
It: 15330, Loss: 8.487e-05, Lambda_1: 5.487, Lambda_2: 1.000000, Time: 0.24
It: 15340, Loss: 1.240e-04, Lambda_1: 5.487, Lambda_2: 1.000000, Time: 0.22
It: 15350, Loss: 8.752e-05, Lambda_1: 5.486, Lambda_2: 1.000000, Time: 0.22
It: 15360, Loss: 9.131e-05, Lambda_1: 5.486, Lambda_2: 1.000000, Time: 0.22
It: 15370, Loss: 8.755e-05, Lambda_1: 5.485, Lambda_2: 1.000000, Time: 0.22
It: 15380, Loss: 8.413e-05, Lambda_1: 5.485, Lambda_2: 1.000000, Time: 0.22
It: 15390, Loss: 8.315e-05, Lambda_1: 5.485, Lambda_2: 1.000000, Time: 0.25
It: 15400, Loss: 8.289e-05, Lambda_1: 5.484, Lambda_2: 1.000000, Time: 0.25
It: 15410, Loss: 8.276e-05, Lambda_1: 5.484, Lambda_2: 1.000000, Time: 0.24
It: 15420, Loss: 8.261e-05, Lambda_1: 5.483, Lambda_2: 1.000000, Time: 0.24
It: 15430, Loss: 8.250e-05, Lambda_1: 5.483, Lambda_2: 1.000000, Time: 0.24
It: 15440, L

It: 16390, Loss: 7.564e-05, Lambda_1: 5.447, Lambda_2: 1.000000, Time: 0.22
It: 16400, Loss: 7.557e-05, Lambda_1: 5.447, Lambda_2: 1.000000, Time: 0.23
It: 16410, Loss: 7.550e-05, Lambda_1: 5.446, Lambda_2: 1.000000, Time: 0.24
It: 16420, Loss: 7.543e-05, Lambda_1: 5.446, Lambda_2: 1.000000, Time: 0.23
It: 16430, Loss: 7.537e-05, Lambda_1: 5.446, Lambda_2: 1.000000, Time: 0.22
It: 16440, Loss: 7.530e-05, Lambda_1: 5.445, Lambda_2: 1.000000, Time: 0.22
It: 16450, Loss: 7.523e-05, Lambda_1: 5.445, Lambda_2: 1.000000, Time: 0.22
It: 16460, Loss: 7.516e-05, Lambda_1: 5.445, Lambda_2: 1.000000, Time: 0.22
It: 16470, Loss: 7.509e-05, Lambda_1: 5.444, Lambda_2: 1.000000, Time: 0.22
It: 16480, Loss: 7.502e-05, Lambda_1: 5.444, Lambda_2: 1.000000, Time: 0.22
It: 16490, Loss: 7.496e-05, Lambda_1: 5.443, Lambda_2: 1.000000, Time: 0.24
It: 16500, Loss: 7.571e-05, Lambda_1: 5.443, Lambda_2: 1.000000, Time: 0.23
It: 16510, Loss: 2.066e-04, Lambda_1: 5.443, Lambda_2: 1.000000, Time: 0.24
It: 16520, L

It: 17470, Loss: 6.927e-05, Lambda_1: 5.412, Lambda_2: 1.000000, Time: 0.22
It: 17480, Loss: 6.922e-05, Lambda_1: 5.411, Lambda_2: 1.000000, Time: 0.22
It: 17490, Loss: 6.916e-05, Lambda_1: 5.411, Lambda_2: 1.000000, Time: 0.24
It: 17500, Loss: 6.910e-05, Lambda_1: 5.411, Lambda_2: 1.000000, Time: 0.23
It: 17510, Loss: 6.904e-05, Lambda_1: 5.411, Lambda_2: 1.000000, Time: 0.22
It: 17520, Loss: 6.899e-05, Lambda_1: 5.410, Lambda_2: 1.000000, Time: 0.22
It: 17530, Loss: 6.918e-05, Lambda_1: 5.410, Lambda_2: 1.000000, Time: 0.22
It: 17540, Loss: 1.161e-04, Lambda_1: 5.410, Lambda_2: 1.000000, Time: 0.22
It: 17550, Loss: 1.390e-04, Lambda_1: 5.409, Lambda_2: 1.000000, Time: 0.22
It: 17560, Loss: 7.298e-05, Lambda_1: 5.409, Lambda_2: 1.000000, Time: 0.22
It: 17570, Loss: 6.907e-05, Lambda_1: 5.409, Lambda_2: 1.000000, Time: 0.23
It: 17580, Loss: 6.908e-05, Lambda_1: 5.408, Lambda_2: 1.000000, Time: 0.23
It: 17590, Loss: 6.924e-05, Lambda_1: 5.408, Lambda_2: 1.000000, Time: 0.25
It: 17600, L

It: 18550, Loss: 6.398e-05, Lambda_1: 5.381, Lambda_2: 1.000000, Time: 0.40
It: 18560, Loss: 6.393e-05, Lambda_1: 5.381, Lambda_2: 1.000000, Time: 0.38
It: 18570, Loss: 6.388e-05, Lambda_1: 5.381, Lambda_2: 1.000000, Time: 0.29
It: 18580, Loss: 6.383e-05, Lambda_1: 5.380, Lambda_2: 1.000000, Time: 0.27
It: 18590, Loss: 6.378e-05, Lambda_1: 5.380, Lambda_2: 1.000000, Time: 0.25
It: 18600, Loss: 6.376e-05, Lambda_1: 5.380, Lambda_2: 1.000000, Time: 0.25
It: 18610, Loss: 6.863e-05, Lambda_1: 5.380, Lambda_2: 1.000000, Time: 0.24
It: 18620, Loss: 2.292e-04, Lambda_1: 5.379, Lambda_2: 1.000000, Time: 0.25
It: 18630, Loss: 8.566e-05, Lambda_1: 5.379, Lambda_2: 1.000000, Time: 0.25
It: 18640, Loss: 6.498e-05, Lambda_1: 5.379, Lambda_2: 1.000000, Time: 0.26
It: 18650, Loss: 6.416e-05, Lambda_1: 5.379, Lambda_2: 1.000000, Time: 0.24
It: 18660, Loss: 6.483e-05, Lambda_1: 5.378, Lambda_2: 1.000000, Time: 0.30
It: 18670, Loss: 6.430e-05, Lambda_1: 5.378, Lambda_2: 1.000000, Time: 0.28
It: 18680, L

It: 19630, Loss: 5.957e-05, Lambda_1: 5.355, Lambda_2: 1.000000, Time: 0.23
It: 19640, Loss: 5.952e-05, Lambda_1: 5.355, Lambda_2: 1.000000, Time: 0.24
It: 19650, Loss: 5.948e-05, Lambda_1: 5.354, Lambda_2: 1.000000, Time: 0.24
It: 19660, Loss: 5.944e-05, Lambda_1: 5.354, Lambda_2: 1.000000, Time: 0.24
It: 19670, Loss: 5.940e-05, Lambda_1: 5.354, Lambda_2: 1.000000, Time: 0.24
It: 19680, Loss: 5.936e-05, Lambda_1: 5.354, Lambda_2: 1.000000, Time: 0.24
It: 19690, Loss: 5.932e-05, Lambda_1: 5.354, Lambda_2: 1.000000, Time: 0.22
It: 19700, Loss: 5.928e-05, Lambda_1: 5.353, Lambda_2: 1.000000, Time: 0.22
It: 19710, Loss: 5.924e-05, Lambda_1: 5.353, Lambda_2: 1.000000, Time: 0.25
It: 19720, Loss: 5.920e-05, Lambda_1: 5.353, Lambda_2: 1.000000, Time: 0.27
It: 19730, Loss: 5.916e-05, Lambda_1: 5.353, Lambda_2: 1.000000, Time: 0.29
It: 19740, Loss: 5.912e-05, Lambda_1: 5.352, Lambda_2: 1.000000, Time: 0.29
It: 19750, Loss: 5.907e-05, Lambda_1: 5.352, Lambda_2: 1.000000, Time: 0.26
It: 19760, L